In [1]:
import random
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import losses
from keras import optimizers
from yaml import safe_load
from typing import List

matplotlib.use('TkAgg')

In [3]:
data = pd.read_csv("../data/cation/DataBase_RF_bi_new_2.csv")

In [4]:
data

,name,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2,S1 #1,S1' #1,S2 #1,S3 #1,S1 #2,S1' #2,S2 #2,S3 #2
0,Structure: Al_15_Si_177_Boulfelfel_Cs_4_K_11,15,177,4,11,1.67,1.38,2.420,0.841,1,1,0,2,2,0,1,0,8,2
1,Structure: Al_15_Si_177_Boulfelfel_Cs_6_K_9,15,177,6,9,1.67,1.38,2.420,0.841,1,1,0,3,2,1,1,0,6,2
2,Structure: Al_15_Si_177_Boulfelfel_K_1_Li_14,15,177,1,14,1.38,0.76,0.841,0.029,1,1,1,0,0,0,0,0,10,4
3,Structure: Al_15_Si_177_Boulfelfel_K_2_Rb_13,15,177,2,13,1.38,1.52,0.841,1.417,1,1,1,0,1,0,0,2,9,2
4,Structure: Al_15_Si_177_Boulfelfel_K_8_Na_7,15,177,8,7,1.38,1.02,0.841,0.181,1,1,1,2,5,0,0,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1950,Structure: Al_96_Si_96_Boulfelfel_Na_78_Rb_18,96,96,78,18,1.02,1.52,0.181,1.417,1,1,0,32,30,16,0,0,2,16
1951,Structure: Al_96_Si_96_Boulfelfel_Rb_1_Na_95,96,96,1,95,1.52,1.02,1.417,0.181,1,1,0,0,0,1,0,32,32,31
1952,Structure: Al_96_Si_96_Boulfelfel_Rb_2_Na_94,96,96,2,94,1.52,1.02,1.417,0.181,1,1,0,0,0,2,0,32,32,30
1953,Structure: Al_96_Si_96_Boulfelfel_Rb_58_K_38,96,96,58,38,1.52,1.38,1.417,0.841,1,1,5,4,19,30,11,3,20,4


In [5]:
data = data.loc[(data["Cation #1"] == (data["S1 #1"] + data["S1\' #1"] + data["S2 #1"] + data["S3 #1"]))
                &
                (data["Cation #2"] == (data["S1 #2"] + data["S1\' #2"] + data["S2 #2"] + data[
                    "S3 #2"]))]  #["Cation #1", "S1 #1", "S1\' #1", "S2 #1", "S3 #1"]]

In [6]:
data_without_names = data.drop(columns=['name'])
plt.matshow(data_without_names.corr())
continuous_features = data_without_names.describe().columns
plt.xticks(range(len(continuous_features)), continuous_features)
plt.yticks(range(len(continuous_features)), continuous_features)
plt.colorbar()
plt.show()

In [7]:
features = data.drop(columns=["S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"],
                     axis=1)
labels = data[["S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"]]

In [8]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.25,
                                                                            random_state=0)

In [9]:
from sklearn.preprocessing import MinMaxScaler
# 
# scale_features_mm = MinMaxScaler()
# features_train = scale_features_mm.fit_transform(features_train)
# features_test = scale_features_mm.transform(features_test)
# 
# scale_labels_mm = MinMaxScaler()
# labels_train = scale_labels_mm.fit_transform(labels_train)
# labels_test = scale_labels_mm.transform(labels_test)

In [51]:
def get_cations(name) -> List[str]:
    cation_list = ["K", "Na", "Cs", "Rb", "Li", "Ca"]
    res = []
    for cation in cation_list:
        if cation in name.values[0]:
            res.append(cation)
    return res


def generate_data_for_set():
    def generate(features):
        num = [[], []]
        x = [[], []]
        cation_type = []
        for i in range(len(features)):
            x[0].append(features[["Cation #1", "Radius #1", "Polarizability #1"]].iloc[i])
            x[1].append(features[["Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
            num[0].append(features[["Cation #1"]].iloc[i])
            num[1].append(features[["Cation #2"]].iloc[i])
            cation_type.append(get_cations(features[["name"]].iloc[i]))
        x[0] = np.array(x[0])
        x[1] = np.array(x[1])
        num[0] = np.array(num[0])
        num[1] = np.array(num[1])
        return x, num, cation_type

    f_train, num_train, cation_type_train = generate(features_train)
    f_test, num_test, cation_type_test = generate(features_test)

    return f_train, f_test, num_train, num_test, cation_type_train, cation_type_test


f_train, f_test, num_train, num_test, cation_type_train, cation_type_test = generate_data_for_set()

# f_train = []
# f_test = []
# num_train = []
# num_test = []
# for i in range(len(features_train)):
#     #labels_train.iloc[i] /= np.sum(labels_train.iloc[i])
#     f_train.append(features_train[["Cation #1", "Radius #1", "Polarizability #1", "Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
#     num_train.append(features_train.iloc[i]["Al"])
# for i in range(len(features_test)):
#     #labels_test.iloc[i] /= np.sum(labels_test.iloc[i])
#     f_test.append(features_test[["Cation #1", "Radius #1", "Polarizability #1", "Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
#     num_test.append(features_test.iloc[i]["Al"])

In [11]:
def mps_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    delta = tf.abs(y_true - y_pred)
    delta_ = tf.reduce_mean(delta)
    sigma = tf.sqrt(tf.reduce_mean(tf.square(delta - delta_)))
    size = tf.size(delta, out_type=tf.int32)
    return tf.reduce_mean(delta_ * (sigma / tf.cast(size, tf.float32) + 1))






def set_model(adapt_data):
    #shared
    inputs = Input(shape=(3,), name="struct_input")
    x = Dense(3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(3, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(3, activation='relu')(x)
    feature_ext_model = Model(inputs, x)
    #
    struct_res = Input(shape=(6,))
    x = Dense(9, activation='relu')(struct_res)
    #x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(4, activation='relu')(x)
    struct_res_model = Model(struct_res, x)
    #
    
    input1, input2 = Input(shape=(3,)), Input(shape=(3,))
    normalization_layer = Normalization()
    normalization_layer.adapt(adapt_data)
    input1 = normalization_layer(input1)
    input2 = normalization_layer(input2)
    num_elements1 = Input(shape=(1,))
    num_elements2 = Input(shape=(1,))
    out1, out2 = feature_ext_model(input1), feature_ext_model(input2)
    y = Average()([out1, out2])
    struct1_state = Concatenate()([y, input1])
    struct2_state = Concatenate()([y, input2])
    struct1_state = struct_res_model(struct1_state)
    struct2_state = struct_res_model(struct2_state)
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct1_state)
    output1 = Multiply()([l1, num_elements1])
    l2 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct2_state)
    output2 = Multiply()([l2, num_elements2])
    out = Concatenate()([output1, output2])
    model = Model([input1, input2, num_elements1, num_elements2], out)
    model.compile(loss='mse', optimizer=optimizers.Adam(clipnorm=0.1))
    return model


import tensorflow as tf

def set_model2(adapt_data):
    #
    struct1, struct2 = Input(shape=3), Input(shape=3)
    normalization_layer = Normalization()
    normalization_layer.adapt(adapt_data)
    struct1 = normalization_layer(struct1)
    struct2 = normalization_layer(struct2)
    inputs = Concatenate()([struct1, struct2])
    x = Dense(6, activation='relu', name="struct_d1")(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation='relu')(x)
    #x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation='relu')(x)
    struct_model = Model([struct1, struct2], x)
    #
    #
    struct_res = Input(shape=(9,))
    x = Dense(9, activation='relu', name="struct_d2")(struct_res)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(9, activation='relu')(x)
    #x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(4, activation='relu')(x)
    struct_res_model = Model(struct_res, x)
    #
    input1, input2 = Input(shape=(3,)), Input(shape=(3,))
    num_elements1 = Input(shape=(1,))
    num_elements2 = Input(shape=(1,))
    out1, out2 = struct_model([input1, input2]), struct_model([input2, input1])
    y = Average()([out1, out2])
    struct1_state = Concatenate()([y, input1])
    struct2_state = Concatenate()([y, input2])
    struct1_state = struct_res_model(struct1_state)
    struct2_state = struct_res_model(struct2_state)
    
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct1_state)
    output1 = Multiply()([l1, num_elements1])
    l2 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct2_state)
    output2 = Multiply()([l2, num_elements2])
    out = Concatenate()([output1, output2])
    
    model = Model([input1, input2, num_elements1, num_elements2], out)
    model.compile(loss=mps_loss, optimizer=optimizers.Adam(clipnorm=0.1)) # losses.Huber(delta=5.0)
    return model



def create_model(x):
    inputs = Input(shape=6)
    num_elements = Input(shape=1)
    x = Dense(10, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    d = Dense(8, activation='relu')(x)
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(d)
    outputs = Multiply()([l1, num_elements])
    model = tf.keras.Model(inputs=[inputs, num_elements], outputs=outputs)
    model.compile(loss="mse", optimizer=optimizers.Adam())
    return model


model = set_model(np.concatenate((f_train[0], f_train[1])))
#model = set_model2(np.concatenate((f_train[0], f_train[1])))

In [12]:
# model.compile(optimizer='adam', loss=mps_loss)
# from keras import backend as K
# K.set_value(model.optimizer.learning_rate, 0.01)
# model.compile(loss=mps_loss, optimizer=optimizers.Adam())

In [19]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()


mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/mps3.keras', save_best_only=True,
                                              monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                              save_freq='epoch')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                      patience=1000, verbose=1, mode='auto')

# history = model.fit([np.array(f_train), np.array(num_train)], labels_train,
#                     epochs=100, batch_size=16, shuffle=True,
#                     validation_data=([np.array(f_test), np.array(num_test)], labels_test), callbacks=[mcp_save])

history = model.fit((f_train[0], f_train[1], num_train[0], num_train[1]), labels_train,
                    epochs=100, batch_size=64, shuffle=True,
                    validation_data=((f_test[0], f_test[1], num_test[0], num_test[1]), labels_test), callbacks=[mcp_save, reduce_lr_loss])

# history = model.fit((f_train[0], f_train[1], num_train[0], num_train[1]), labels_train,
#                     epochs=500, batch_size=64, shuffle=True,
#                     validation_data=((f_ref[0], f_ref[1], num_ref[0], num_ref[1]), labels_ref), callbacks=[mcp_save, reduce_lr_loss])

plot_loss(history)

Epoch 1/100
 1/23 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 15.2781
Epoch 1: val_loss improved from inf to 15.26992, saving model to data/models/mps3.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16.2912 - val_loss: 15.2699 - learning_rate: 0.0010
Epoch 2/100
 1/23 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 17.3206
Epoch 2: val_loss improved from 15.26992 to 15.17541, saving model to data/models/mps3.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 17.3516 - val_loss: 15.1754 - learning_rate: 0.0010
Epoch 3/100
 1/23 ━━━━━━━━━━━━━━━━━━━━ 4s 187ms/step - loss: 15.2370
Epoch 3: val_loss improved from 15.17541 to 15.11679, saving model to data/models/mps3.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 16.9325 - val_loss: 15.1168 - learning_rate: 0.0010
Epoch 4/100
 1/23 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 18.4066
Epoch 4: val_loss improved from 15.11679 to 15.03265, saving model to data/models/mps3.keras
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 16.3466 - val_lo

In [37]:
model = tf.keras.models.load_model('data/models/mps3.keras', custom_objects={'mps_loss': mps_loss}, safe_mode=False)

ValueError: Layer node index out of bounds.
inbound_layer = <InputLayer name=keras_tensor_167CLONE, built=True>
inbound_layer._inbound_nodes = [<Node operation=keras_tensor_167CLONE, id=2012069611744>]
inbound_node_index = 1

In [20]:
def process_prediction(prediction, num1, num2):
    return np.concatenate((prediction[:4] * num1, prediction[4:] * num2))


#predictions = model.predict([np.array(f_test), np.array(num_test)])
predictions = model.predict((f_test[0], f_test[1], num_test[0], num_test[1]))
predictions_2 = model.predict((f_test[1], f_test[0], num_test[1], num_test[0]))

#predictions = [process_prediction(predictions[i], num_test[0][i], num_test[1][i]) for i in range(len(predictions))]
#labels_test = [process_prediction(labels_test[i], num_test[0][i], num_test[1][i]) for i in range(len(labels_test))]

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [21]:
print(features_test.iloc[i])
print(labels_test.iloc[i])

name                 Structure: Al_92_Si_100_Boulfelfel_Li_35_Na_57
Al                                                               92
Si                                                              100
Cation #1                                                        35
Cation #2                                                        57
Radius #1                                                      0.76
Radius #2                                                      1.02
Polarizability #1                                             0.029
Polarizability #2                                             0.181
Charge #1                                                         1
Charge #2                                                         1
Name: 1846, dtype: object
S1 #1      0
S1' #1    11
S2 #1      6
S3 #1     18
S1 #2      2
S1' #2    17
S2 #2     26
S3 #2     12
Name: 1846, dtype: int64


In [22]:
def mps(first:np.array, second:np.array):
    delta = np.abs(first - second)
    delta_ = np.mean(delta)
    sigma = np.sqrt(np.mean(np.abs(delta - delta_)))
    N = len(delta)
    return np.mean(delta_/N * (sigma/N + 1))



i = random.randint(0, len(predictions) - 1)
print("№", i)
rounded_prediction = [round(p, 1) for p in predictions[i]]
print(rounded_prediction)
rounded_prediction2 = [round(p, 1) for p in predictions_2[i]]
#print(rounded_prediction2)

rounded_real = [round(float(p), 1) for p in np.array(labels_test.iloc[i])]
print(rounded_real)
print(sum(predictions[i]), sum(labels_test.iloc[i]))

№ 327
[2.3, 3.4, 15.0, 13.3, 1.3, 2.2, 7.4, 2.1]
[0.0, 2.0, 21.0, 11.0, 4.0, 1.0, 8.0, 0.0]
47.00000238418579 47


In [60]:
mps_list = []
for i in range(len(predictions)):
    p = predictions[i]
    real = np.array(labels_test.iloc[i])
    mps_list.append(mps(p, real))
print(sum(mps_list))
print(sum(mps_list)/len(mps_list))

145.1071169545295
0.29918993186500925


In [61]:
plt.hist(mps_list)
plt.show()

In [65]:
plt.scatter([str(i) for  i in cation_type_test], mps_list)
plt.show()

In [12]:
# extended_data = pd.DataFrame(columns=["Al", "Si", "Cation #1", "Cation #2", "Radius #1", "Radius #2", "Polarizability #1", "Polarizability #2",
#                                       "Charge #1", "Charge #2", "S1 #1", "S1\' #1", "S2 #1", "S3 #1",
#                                       "S1 #2", "S1\' #2", "S2 #2", "S3 #2"])
for index, row in data.iterrows():
    data.loc[len(data)] = [row["name"], row["Al"], row["Si"],
                                             row["Cation #2"], row["Cation #1"],
                                             row["Radius #2"], row["Radius #1"],
                                             row["Polarizability #2"], row["Polarizability #1"],
                                             row["Charge #2"], row["Charge #1"],
                                             row["S1 #2"], row["S1\' #2"], row["S2 #2"], row["S3 #2"],
                                             row["S1 #1"], row["S1\' #1"], row["S2 #1"], row["S3 #1"]]
data = data.drop(columns=["name"])

In [16]:
from cation_problem.corr_plot import corr_heatmap_with_pval
corr_heatmap_with_pval(data, method ='kendall', figsize=(10, 5))

In [53]:
len(data)

1000

In [52]:
reference_data = pd.read_csv('data/cation/Reference_Data_Base_2024_fp.csv', delimiter=',', header=None,)
reference_data = reference_data.T
new_header = reference_data.iloc[0]
reference_data = reference_data[1:]
reference_data.columns = new_header
reference_data

FileNotFoundError: [Errno 2] No such file or directory: 'data/cation/Reference_Data_Base_2024_fp.csv'

In [46]:
bi_cation_data = reference_data.loc[(reference_data["Cation #1"].notnull()) & (reference_data["Cation #1"] != "-") &
                          (reference_data["Cation #2"].notnull()) & (reference_data["Cation #2"] != "-") &
                          (reference_data["Cation #3"].isnull() | (reference_data["Cation #3"] == "-"))]
bi_cation_data

,Cation #1,Number of Cations #1 per UC,Charge of Cation #1,Cation #2,Number of Cations #2 per UC,Charge of Cation #2,Cation #3,Number of cations #3 per UC,Charge of Cation #3,Number of Al atoms per UC,...,Charge balance (True/False),Data source (NMR/XRD),Dehydrated (yes/no),Dehydration temperature (in K),Evacuated (yes/no),Evacuation pressure (in Torr),Crystallography structure (yes/no),Average UC length (in Angstroms),Doi:,Comments
16,K,52,1,Na,39,1,NaN,NaN,NaN,91.0,...,True,XRD,yes,673,no,760,NaN,24.861,https://doi.org/10.1016/s0144-2449(05)80234-9,NaN
17,Cs,45,1,Na,47,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,673,yes,2.00E-6,yes,24.898,https://doi.org/10.1016/j.micromeso.2004.01.006,NaN
19,Co,38.1,2,Na,15.8,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,673,yes,1.00E-3,yes,24.597,https://doi.org/10.1016/j.micromeso.2007.12.031,NaN
20,Cs,45.9,1,Na,37.6,1,NaN,NaN,NaN,83.5,...,True,NMR,yes,673,yes,7.50E-5,no,NaN,https://doi.org/10.1021/jp001571g,NaN
21,Ca,19,2,Na,48,1,NaN,NaN,NaN,86.0,...,True,XRD,yes,773,yes,NaN,no,NaN,https://doi.org/10.1016/j.micromeso.2007.01.043,NaN
24,Co,34,2,Ca,9,2,NaN,NaN,NaN,86.0,...,True,XRD,yes,773,no,NaN,NaN,NaN,https://doi.org/10.1016/j.micromeso.2007.01.043,NaN
34,Cs,44,1,Na,47,1,NaN,NaN,NaN,91.0,...,True,XRD,yes,673,no,7.60E+2,yes,24.98,10.1016/s0144-2449(05)80234-9,NaN
36,Li,86,1,Na,6,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,623,yes,1.00E-6,NaN,NaN,https://doi.org/10.7745/KJSSF.2013.46.4.260,NaN
37,Li,87,1,Na,5,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,623,yes,1.00E-6,NaN,NaN,https://doi.org/10.7745/KJSSF.2013.46.4.261,NaN
38,Li,81,1,Na,15,1,NaN,NaN,NaN,96.0,...,True,NMR,yes,296,NaN,7.50E-5,NaN,NaN,https://doi.org/10.1016/S1387-1811(98)00211-X,NaN


In [47]:
cation_db = pd.read_csv('data/cation/cations_params.csv')
cation_db

FileNotFoundError: [Errno 2] No such file or directory: 'data/cation/cations_params.csv'

In [174]:
def generate_data_for_reference(data_ref: pd.DataFrame, cation_db: pd.DataFrame):
    num_ref = [[], []]
    f_ref = [[], []]
    labels_ref = []
    cations_type = []
    
    for index, row in data_ref.iterrows():
        cation_number1 = float(row["Number of Cations #1 per UC"])
        cation_number2 = float(row["Number of Cations #2 per UC"])
        num_ref[0].append(cation_number1)
        num_ref[1].append(cation_number2)
        radius1 = cation_db[cation_db["Cation"] == row["Cation #1"]]["Radius"].values[0]
        radius2 = cation_db[cation_db["Cation"] == row["Cation #2"]]["Radius"].values[0]
        polarizability1 = cation_db[cation_db["Cation"] == row["Cation #1"]]["Polarizability"].values[0]
        polarizability2 = cation_db[cation_db["Cation"] == row["Cation #2"]]["Polarizability"].values[0]
        f_ref[0].append([cation_number1, radius1, polarizability1])
        f_ref[1].append([cation_number2, radius2, polarizability2])
        
        labels_ref.append([row["SI site occupation by Cation #1"], row["SI\' site occupation by Cation #1"],
                           row["SII site occupation by Cation #1"], row["SIII site occupation by Cation #1"],
                           
                           row["SI site occupation by Cation #2"], row["SI\' site occupation by Cation #2"],
                           row["SII site occupation by Cation #2"], row["SIII site occupation by Cation #2"]])
        cations_type.append([[row["Cation #1"]], [row["Cation #2"]]])
    f_ref[0] = np.array(f_ref[0])
    f_ref[1] = np.array(f_ref[1])
    num_ref[0] = np.array(num_ref[0])
    num_ref[1] = np.array(num_ref[1])
    labels_ref = np.array(labels_ref)
    return f_ref, num_ref, labels_ref, cations_type

f_ref, num_ref, labels_ref, cations_type = generate_data_for_reference(bi_cation_data, cation_db)
labels_ref = labels_ref.astype(float)

In [168]:
cation = bi_cation_data.iloc[1]["Cation #1"]
print(cation)
print(cation_db[cation_db["Cation"] == cation]["Radius"].values[0])
print(f_ref[0][1])

Cs
1.67
[45.    1.67  2.42]


In [220]:
predictions_ref = model.predict((f_ref[0], f_ref[1], num_ref[0], num_ref[1]))

2/2 [==============================] - 2s 129ms/step


In [225]:
i = random.randint(0, len(predictions_ref) - 1)
print("№", i)
rounded_prediction = [round(p, 1) for p in predictions_ref[i]]
print(rounded_prediction)

rounded_real = [round(float(p), 1) for p in np.array(labels_ref[i])]
print(rounded_real)
print(sum(predictions_ref[i]), sum(labels_ref[i]))
print("mps: ", mps(predictions_ref[i], labels_ref[i]))
print(f_ref[0][i])
print(cations_type[i])

№ 31
[2.4, 2.5, 13.0, 5.2, 13.5, 6.6, 30.6, 18.3]
[15.7, 0.0, 7.4, 0.0, 0.0, 17.7, 24.0, 27.2]
91.99999952316284 92.0
mps:  1.2775419515186626
[23.    1.15  1.36]
[['Ag'], ['Tl']]


In [222]:
mps_list = []
for i in range(len(predictions_ref)):
    p = predictions_ref[i]
    real = np.array(labels_ref[i])
    mps_list.append(mps(p, real))
print(sum(mps_list))
print(sum(mps_list)/len(mps_list))
plt.hist(mps_list)
plt.show()

42.82590133481233
0.8397235555845555
